In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt

In [ ]:
class ChessBoard:
    def __init__(self,cols,rows,size):
        self.cols = cols-1
        self.rows = rows-1
        self.size = size
        self.objPoints()
    def objPoints(self):
        c = self.cols
        r = self.rows
        dim =3
        self.obj_p = np.zeros((c*r,dim),np.float32)
        self.obj_p[:,:2] = np.mgrid[0:c,0:r].T.reshape(-1,2)
        self.obj_p = self.obj_p * self.size
        
# チェッカーボードの大きさを設定
board = ChessBoard(10,7,24.0)
print('BOARD_INFO')
print('|- CORNER_COLS :',board.cols)
print('|- CORNER_ROWS :',board.rows)
print('|- PATTERN_SIZE:',board.size)
# print(board.obj_p)

In [ ]:
# read images (recommend 10~15 images)
im_w = 640
im_h = 480
Images = np.empty((0,im_h,im_w,3),dtype=np.uint8)
imgs_name = glob.glob('sample_images/*.png')
imgs_num = len(imgs_name)
for i in imgs_name:
    img = cv2.imread(i)
    if(img is None):
        print('imread failed:', i)
        continue
    print(i)
    '''
    img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.show()
    '''
    img = np.expand_dims(img,axis=0)
    Images = np.vstack((Images,img))
imgs__num = len(Images)
print(imgs_num,'IMAGES')

In [ ]:
# detect chessboard corners
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
ImgPoints = []
ObjPoints = []
for i in range(len(Images)):
    img = Images[i].copy()
    img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(img_gray,(board.cols,board.rows))
    if ret:
        corners2 = cv2.cornerSubPix(img_gray,corners,(11,11),(-1,-1),criteria)
        img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        cv2.drawChessboardCorners(img_rgb, (board.cols,board.rows), corners2[0], ret)
        plt.imshow(img_rgb)
        plt.show()
        ImgPoints.append(corners2)
        ObjPoints.append(board.obj_p)
    else:
        print('corners detection failed.')
print('ImgPoints',len(ImgPoints))
print('ObjPoints',len(ObjPoints))

In [ ]:
# calibrate
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(ObjPoints, ImgPoints,(im_h,im_w),None,None,None,None,cv2.CALIB_RATIONAL_MODEL)

In [ ]:
print(mtx)
print(dist)

In [ ]:
fx = mtx[0,0]/im_w
fy = mtx[1,1]/im_h
cx = mtx[0,2]/im_w
cy = mtx[1,2]/im_h
k0 = dist[0,0]
k1 = dist[0,1]
k2 = dist[0,2]
k3 = dist[0,3]
k4 = dist[0,4]
k5 = dist[0,5]
k6 = dist[0,6]
k7 = dist[0,7]

print('fx {:.7f}'.format(fx))
print('fy {:.7f}'.format(fy))
print('cx {:.7f}'.format(cx))
print('cy {:.7f}'.format(cy))
print('k0 {:.7f}'.format(k0))
print('k1 {:.7f}'.format(k1))
print('k2 {:.7f}'.format(k2))
print('k3 {:.7f}'.format(k3))
print('k4 {:.7f}'.format(k4))
print('k5 {:.7f}'.format(k5))
print('k6 {:.7f}'.format(k6))
print('k7 {:.7f}'.format(k7))

In [ ]:
rand = np.random.randint(imgs_num)
img = Images[1].copy()
img_und = cv2.undistort(img,mtx,dist)
img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img_und_rgb = cv2.cvtColor(img_und,cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb)
plt.show()
plt.imshow(img_und_rgb)
plt.show()